# Streamflow normalization. 

KazHydroMet initially collects streamflow data in [m^3/s]. Ensuring the NeuralHydrology guidelines, 
streamflow (discharge) values should be [mm/day], which could be done by area normalization.

In [ ]:
import pandas as pd 
import os 
import numpy as np 
import seaborn as sns 
import matplotlib.pyplot as plt

In [ ]:
area_values = pd.read_csv("")  # define your path
# area_values.rename(columns={'Catchment_ID': "basin_id", "Area_sq_km": "area_gages2"}, inplace=True)
area_values

In [ ]:
area_values['basin_id'] = area_values['basin_id'].apply(lambda x: x.split('/')[-1])
area_values

In [ ]:
area_values['basin_id'] = area_values['basin_id'].apply(lambda x: str(x))
area_values

Area values are ready for the normalization

In [ ]:
path1 = ""                  # define your path, for sigma dataset 
path2 = ""                  # define your path, for omega dataset

In [ ]:
import shutil

# Move the file
# shutil.move(source_file, destination_dir)

In [ ]:
for file in os.listdir(path1):

    if (file not in os.listdir(path2)):
        print("This file is not in Yearbooks: " + file)
        shutil.copy(path1 + file, 'omega/' + file)
        continue
    
    khm = pd.read_csv(path1 + file)
    llama = pd.read_csv(path2 + file)

    # Merge on date, keep all dates
    merged = pd.merge(khm, llama, on='date', how='outer', suffixes=('_khm', '_llama'))

    # Fill values: use Llama Values if not NaN, else use file2
    merged['discharge'] = merged['discharge_llama'].combine_first(merged['discharge_khm'])

    # Keep only date and merged value
    merged = merged[['date', 'discharge']]

    # Filter by date range
    filtered = merged[(merged['date'] >= '2000-01-01') & (merged['date'] <= '2022-12-31')]

    filtered.to_csv('omega/' + file, index=False)
    print(f'this file {file} is done\n')


In [ ]:
for file in os.listdir(path2):
    if file not in os.listdir(path1): 
        print(file)
        shutil.copy(path2 + file, 'omega/' + file)

In [ ]:
area = area_values.to_dict('split')
temp = dict(area['data'])


In [ ]:
for file in os.listdir('omega/'):
    if (file in os.listdir('omega_norm/')):
        continue
    tempo = pd.read_csv('omega/' + file)
    tempo['discharge'] = tempo['discharge'].replace('', np.nan)
    tempo['discharge'] = tempo['discharge'].astype(float)
    tempo.to_csv('omega_norm/' + file, index=False)
    print(file + " is done \n")

In [ ]:
for file in os.listdir('omega_norm/'):
    tempo = pd.read_csv('omega_norm/' + file)
    tempo['discharge'] = (tempo['discharge'].astype(float) * 86.4) / temp[file.split('.')[0]]
    tempo.to_csv('omega_norm/' + file, index=False)
    print(file + " is done \n")